In [11]:
import os
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

import tensorflow as tf
from tensorflow.keras.utils import to_categorical, Sequence
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.applications import InceptionV3
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report


In [12]:
# -------------------- Config -------------------- #
img_size = 224
batch_size = 16
num_classes = 20
train_paths = [
    "../input/socofing/SOCOFing/Altered/Altered-Easy",
    "../input/socofing/SOCOFing/Altered/Altered-Medium",
    "../input/socofing/SOCOFing/Altered/Altered-Hard"
]
test_path = "../input/socofing/SOCOFing/Real"


In [13]:
def extract_label(img_path):
    filename = os.path.basename(img_path)
    name, _ = os.path.splitext(filename)
    try:
        _, info = name.split('__')
    except ValueError:
        raise ValueError(f"Filename format unexpected: {filename}")
    parts = info.split('_')
    if len(parts) < 4:
        raise ValueError(f"Filename format incomplete: {filename}")
    gender = 'Male' if parts[0].upper() == 'M' else 'Female'
    lr = 'Left' if parts[1].lower() == 'left' else 'Right'
    finger = parts[2].lower()
    return f"{gender}_{lr}_{finger}"

def get_paths_and_labels(folder_paths):
    img_paths = []
    labels = []
    for folder in folder_paths:
        for img in os.listdir(folder):
            path = os.path.join(folder, img)
            try:
                label = extract_label(path)
                img_paths.append(path)
                labels.append(label)
            except Exception as e:
                print(f"Failed for {img}: {e}")
    return img_paths, labels

# Load training and test paths
X_train_paths, y_train_labels = get_paths_and_labels(train_paths)
X_test_paths, y_test_labels = get_paths_and_labels([test_path])


In [14]:
all_labels = sorted(list(set(y_train_labels + y_test_labels)))
label_to_index = {label: idx for idx, label in enumerate(all_labels)}
index_to_label = {idx: label for label, idx in label_to_index.items()}

# Encode labels
y_train = np.array([label_to_index[label] for label in y_train_labels])
y_test = np.array([label_to_index[label] for label in y_test_labels])

# Balance classes in training
train_dict = {}
for path, label_idx in zip(X_train_paths, y_train):
    if label_idx not in train_dict:
        train_dict[label_idx] = []
    train_dict[label_idx].append(path)

min_count = min(len(v) for v in train_dict.values())
balanced_X_train = []
balanced_y_train = []

for label_idx, paths in train_dict.items():
    sampled = random.sample(paths, min_count)
    balanced_X_train.extend(sampled)
    balanced_y_train.extend([label_idx]*len(sampled))

X_train_paths = balanced_X_train
y_train = np.array(balanced_y_train)

# Train-validation split
X_train_paths, X_val_paths, y_train, y_val = train_test_split(
    X_train_paths, y_train, test_size=0.2, stratify=y_train, random_state=42
)


In [15]:
class FingerDataGenerator(Sequence):
    def __init__(self, img_paths, labels, batch_size=16, img_size=224, num_classes=20, shuffle=True):
        self.img_paths = img_paths
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.num_classes = num_classes
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.img_paths))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.img_paths)/self.batch_size))

    def __getitem__(self, index):
        batch_indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        batch_paths = [self.img_paths[i] for i in batch_indexes]
        batch_labels = [self.labels[i] for i in batch_indexes]
        X = np.zeros((len(batch_paths), self.img_size, self.img_size, 3), dtype=np.float32)
        y = np.zeros((len(batch_paths), self.num_classes), dtype=np.float32)
        for i, path in enumerate(batch_paths):
            img = cv2.imread(path)
            img = cv2.resize(img, (self.img_size, self.img_size))
            img = img / 255.0
            X[i] = img
            y[i] = to_categorical(batch_labels[i], num_classes=self.num_classes)
        return X, y

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

# Generators
train_generator = FingerDataGenerator(X_train_paths, y_train, batch_size=batch_size, img_size=img_size, num_classes=num_classes)
val_generator = FingerDataGenerator(X_val_paths, y_val, batch_size=batch_size, img_size=img_size, num_classes=num_classes, shuffle=False)
test_generator = FingerDataGenerator(X_test_paths, y_test, batch_size=batch_size, img_size=img_size, num_classes=num_classes, shuffle=False)


In [24]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense

# Base model
base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=Input(shape=(img_size,img_size,3)))

# Phase 1: Freeze all layers
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_94 (Conv2D)  │ (None, 111, 111,  │        864 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 111, 111,  │         96 │ conv2d_94[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_94       │ (None, 111, 111,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_95 (Conv2D)  │ (None, 109, 109,  │      9,216 │ activation_94[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │         96 │ conv2d_95[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_95       │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_96 (Conv2D)  │ (None, 109, 109,  │     18,432 │ activation_95[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │        192 │ conv2d_96[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_96       │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_4     │ (None, 54, 54,    │          0 │ activation_96[0]… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_97 (Conv2D)  │ (None, 54, 54,    │      5,120 │ max_pooling2d_4[… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 54, 54,    │        240 │ conv2d_97[0][0]   │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_97       │ (None, 54, 54,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_98 (Conv2D)  │ (None, 52, 52,    │    138,240 │ activation_97[0]… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 52, 52,    │        576 │ conv2d_98[0][0]   │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_98       │ (None, 52, 52,    │          0 │ batch_normalizat

 Total params: 21,843,764 (83.33 MB)

 Trainable params: 40,980 (160.08 KB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [26]:
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# ---------------- Class weights ---------------- #
# y_train should be integer class indices (0-19)
class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = dict(enumerate(class_weights_array))
print("Class weights:", class_weights)

# ---------------- Callbacks ---------------- #
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3),
    ModelCheckpoint("/kaggle/working/finger_inceptionv3.h5", monitor='val_loss', save_best_only=True)
]

# ---------------- Phase 1: Freeze base model ---------------- #
for layer in base_model.layers:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),  # initial learning rate
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_phase1 = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,       # You can increase this
    class_weight=class_weights,
    callbacks=callbacks
)

# ---------------- Phase 2: Fine-tune ---------------- #
# Unfreeze last N layers (e.g., 100) for more fine-tuning
for layer in base_model.layers[-100:]:
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),  # smaller learning rate for fine-tuning
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_phase2 = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,       # You can increase this
    class_weight=class_weights,
    callbacks=callbacks
)


Class weights: {0: 0.9994722955145119, 1: 0.9994722955145119, 2: 0.9994722955145119, 3: 0.9994722955145119, 4: 1.000792602377807, 5: 0.9994722955145119, 6: 1.000792602377807, 7: 1.000792602377807, 8: 0.9994722955145119, 9: 1.000792602377807, 10: 1.000792602377807, 11: 0.9994722955145119, 12: 1.000792602377807, 13: 0.9994722955145119, 14: 1.000792602377807, 15: 0.9994722955145119, 16: 0.9994722955145119, 17: 0.9994722955145119, 18: 0.9994722955145119, 19: 1.000792602377807}
Epoch 1/15


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_314']
Received: inputs=Tensor(shape=(None, 224, 224, 3))
  warnings.warn(msg)


947/947 ━━━━━━━━━━━━━━━━━━━━ 74s 62ms/step - accuracy: 0.1177 - loss: 3.1773 - val_accuracy: 0.2175 - val_loss: 2.4470 - learning_rate: 0.0010
Epoch 2/15
947/947 ━━━━━━━━━━━━━━━━━━━━ 37s 39ms/step - accuracy: 0.2094 - loss: 2.6394 - val_accuracy: 0.2421 - val_loss: 2.4543 - learning_rate: 0.0010
Epoch 3/15
947/947 ━━━━━━━━━━━━━━━━━━━━ 38s 40ms/step - accuracy: 0.2285 - loss: 2.5489 - val_accuracy: 0.2487 - val_loss: 2.3616 - learning_rate: 0.0010
Epoch 4/15
947/947 ━━━━━━━━━━━━━━━━━━━━ 36s 38ms/step - accuracy: 0.2384 - loss: 2.5011 - val_accuracy: 0.2685 - val_loss: 2.2251 - learning_rate: 0.0010
Epoch 5/15
947/947 ━━━━━━━━━━━━━━━━━━━━ 36s 38ms/step - accuracy: 0.2541 - loss: 2.4546 - val_accuracy: 0.2888 - val_loss: 2.2194 - learning_rate: 0.0010
Epoch 6/15
947/947 ━━━━━━━━━━━━━━━━━━━━ 37s 39ms/step - accuracy: 0.2602 - loss: 2.4390 - val_accuracy: 0.2822 - val_loss: 2.3038 - learning_rate: 0.0010
Epoch 7/15
947/947 ━━━━━━━━━━━━━━━━━━━━ 36s 38ms/step - accuracy: 0.2583 - loss: 2.4756

In [ ]:
# Evaluate
train_loss, train_acc = model.evaluate(train_generator)
val_loss, val_acc = model.evaluate(val_generator)
test_loss, test_acc = model.evaluate(test_generator)

print(f"Training Accuracy: {train_acc*100:.2f}%")
print(f"Validation Accuracy: {val_acc*100:.2f}%")
print(f"Test Accuracy: {test_acc*100:.2f}%")

# Confusion Matrix
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(np.vstack([y for x, y in test_generator]), axis=1)

cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(12,10))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=all_labels, yticklabels=all_labels, cmap='Blues')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

# Classification Report
print(classification_report(y_true, y_pred_classes, target_names=all_labels))


947/947 ━━━━━━━━━━━━━━━━━━━━ 41s 44ms/step - accuracy: 1.0000 - loss: 0.0063
237/237 ━━━━━━━━━━━━━━━━━━━━ 10s 43ms/step - accuracy: 0.7269 - loss: 0.8908


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


375/375 ━━━━━━━━━━━━━━━━━━━━ 50s 134ms/step - accuracy: 0.7929 - loss: 0.6924
Training Accuracy: 99.99%
Validation Accuracy: 72.15%
Test Accuracy: 79.03%


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_314']
Received: inputs=Tensor(shape=(16, 224, 224, 3))
  warnings.warn(msg)


375/375 ━━━━━━━━━━━━━━━━━━━━ 18s 32ms/step
